In [13]:
import pandas as pd
import os
import gzip
import spacy as sp
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from collections import defaultdict
from sklearn.cluster import DBSCAN
from math import log
from nltk.corpus import wordnet as wn
import nltk
from itertools import product
import pickle
import warnings
warnings.filterwarnings('ignore')

In [14]:
nlp = sp.load('en')

In [15]:
# Original metaphors loading

eval_data = pd.read_csv("eval-mod_ours.txt")

metaphor_words = []
metaphors = {}

for r in eval_data.iterrows():
    #tokens = nlp(r[1]["Metaphor"].decode('utf-8', errors='ignore').strip().lower())
    tokens = nlp(r[1]["Metaphor"].strip().lower())
    topic = tokens[0]
    vehicle = tokens[-1]
    if topic.lemma_ not in metaphor_words:
        metaphor_words.append(topic.lemma_)
    if vehicle.lemma_ not in metaphor_words:
        metaphor_words.append(vehicle.lemma_)
    if r[1]["Metaphor"] not in metaphors:
        metaphors[r[1]["Metaphor"]] = [topic.lemma_, vehicle.lemma_, 
                                       [(str(r[1]["Interpretation"]).lower(), r[1]["Freq"])],
                                       [str(r[1]["Interpretation"]).lower()]
                                      ]
    else:
        metaphors[r[1]["Metaphor"]][2].append((str(r[1]["Interpretation"]).lower(), r[1]["Freq"]))
        metaphors[r[1]["Metaphor"]][3].append(str(r[1]["Interpretation"]).lower())


In [49]:
# Loading Lena's new metaphors

metaphor_words = []
with open("lena_metaphors_individual_words.txt", "rt") as f:
    for line in f.readlines():
        metaphor_words.append(line.strip())


print(metaphor_words)

['man', 'ox', 'body', 'spring', 'face', 'white', 'deathmask', 'sensation', 'fever', 'face', 'land', 'ghost', 'women', 'goddess', 'pain', 'knife', 'plate', 'voice', 'shower', 'rocks', 'man', 'zombie', 'body', 'armor', 'suit', 'man', 'bull', 'man', 'pitbull', 'pit-bull', 'place', 'tomb', 'destruction', 'firework', 'swamp', 'cemetery', 'pain', 'bomb', 'life', 'chemistry', 'experiment', 'man', 'gorilla', 'place', 'museum', 'i', 'whale', 'voice', 'feather', 'body', 'bowstring', 'bird', 'marble', 'thunder', 'diamond', 'rough', 'waterfall', 'geezer', 'happiness', 'fever', 'eyes', 'steel', 'music', 'heart', 'women', 'angel', 'bottomless', 'pits', 'water', 'man', 'marble', 'statue', 'man', 'horse', 'place', 'museum', 'answer', 'viper', 'hissing', 'man', 'animal', 'man', 'rock', 'love', 'oxygen', 'apartment', 'meat', 'locker', 'culture', 'food', 'fox', 'neck', 'tree', 'trunk', 'life', 'desert', 'robot', 'cheetah', 'mule', 'wallflower', 'plan', 'house', 'cards', 'words', 'slap', 'office', 'hive',

In [17]:
class Lang:
    def __init__(self):
        self.vec = []
        self.word_count = 0
        self.ind2word = {}
        self.word2ind = {}
        for line in open("/Users/Kfir/Documents/corpora/glove.6B/glove.6B.300d.txt"):
            values = line.split(" ")
            v = []
            for i in range (1, len(values)):
                v.append(float(values[i]))
            self.vec.append(v)
            self.ind2word[self.word_count] = values[0]
            self.word2ind[values[0]] = self.word_count
            self.word_count += 1
    
    def get_vec(self, word):
        word = word.strip().lower()
        if word in self.word2ind:
            return self.vec[self.word2ind[word]]
        return None
            
lang = Lang()

In [50]:
class DepNode:
    def __init__(self, line):
        tokens = line.split('\t')
        self.id = int(tokens[0])
        self.text = tokens[1].lower()
        self.lemma = tokens[2].lower()
        self.pos = tokens[3]
        self.head = int(tokens[6])
        self.dep_type = tokens[7]
        self.children = []

class DepSentence:
    def __init__(self):
        self.nodes = []
        
    def add_word(self, line):
        node = DepNode(line)
        self.nodes.append(node)
        return node
        
    def rewire(self):
        for n in self.nodes:
            if n.dep_type != 'ROOT':
                self.nodes[n.head].children.append(n)
    

def is_valid_meaning(p):
    #if self.pos in ['JJ', 'JJS', 'RB', 'VBG']:
    #if p in ['JJ', 'JJS', 'VBG']:
    if p in ['VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'VB']:
        return True
    return False
    

def count_words(dep_corpus_folder):
    words = 0
    files = [f for f in os.listdir(dep_corpus_folder) if os.path.isfile(os.path.join(dep_corpus_folder, f))]
    for f in files:
        full_path = os.path.join(dep_corpus_folder, f)
        print("processing", full_path)
        sentence = None
        valid = False
        for line in gzip.open(full_path):
            if len(line.strip()) > 0:
                words += 0
                
    print(words)
        
def find_candidates_for(words, dep_corpus_folder):
    candidates = {}
    files = [f for f in os.listdir(dep_corpus_folder) if os.path.isfile(os.path.join(dep_corpus_folder, f))]
    for f in files:
        full_path = os.path.join(dep_corpus_folder, f)
        print("processing", full_path)
        sentence = None
        valid = False
        for line in gzip.open(full_path):
            line = line.decode('utf-8')
            line = line.strip()
            if len(line) > 0 and line[0] != '#':
                if sentence is None:
                    sentence = DepSentence()
                node = sentence.add_word(line)
                if node.text in words:
                    valid = True
            elif sentence is not None:
                if valid:
                    sentence.rewire()
                    for n in sentence.nodes:
                        if n.text in words:
                            if n.text not in candidates:
                                candidates[n.text] = Counter()
                            if n.dep_type != 'ROOT' and is_valid_meaning(sentence.nodes[n.head].pos):
                                candidates[n.text][sentence.nodes[n.head].text] += 1
                            for c in n.children:
                                if is_valid_meaning(c.pos):
                                    candidates[n.text][c.text] += 1
                sentence = None
    return candidates

In [55]:
candidates = find_candidates_for(metaphor_words, 'dep_corpus')

processing dep_corpus/part-m-00012.gz
processing dep_corpus/part-m-00026.gz
processing dep_corpus/part-m-00036.gz
processing dep_corpus/part-m-00002.gz
processing dep_corpus/part-m-00022.gz
processing dep_corpus/part-m-00006.gz
processing dep_corpus/part-m-00032.gz
processing dep_corpus/part-m-00023.gz
processing dep_corpus/part-m-00017.gz
processing dep_corpus/part-m-00007.gz
processing dep_corpus/part-m-00033.gz
processing dep_corpus/part-m-00027.gz
processing dep_corpus/part-m-00037.gz
processing dep_corpus/part-m-00003.gz
processing dep_corpus/part-m-00018.gz
processing dep_corpus/part-m-00008.gz
processing dep_corpus/part-m-00028.gz
processing dep_corpus/part-m-00038.gz
processing dep_corpus/part-m-00029.gz
processing dep_corpus/part-m-00039.gz
processing dep_corpus/part-m-00019.gz
processing dep_corpus/part-m-00009.gz
processing dep_corpus/part-m-00020.gz
processing dep_corpus/part-m-00014.gz
processing dep_corpus/part-m-00004.gz
processing dep_corpus/part-m-00030.gz
processing d

In [24]:
pickle.dump(candidates, open( "candidates-lena-metaphors.p", "wb" ) )

In [54]:
print(candidates['armor'].most_common(200))

[('restricts', 2), ('accept', 2), ('are', 2), ('goes', 2), ('wearing', 2), ('have', 2), ('breaks', 2), ('take', 2), ('lose', 1), ('loses', 1), ('equip', 1), ('put', 1), ('called', 1), ('survive', 1), ('obtain', 1), ('mitigates', 1), ('has', 1), ('sacrifice', 1), ('come', 1), ('earn', 1), ('wear', 1), ('got', 1), ('gaining', 1), ('means', 1), ('grants', 1), ('added', 1), ('was', 1), ('is', 1), ('calls', 1), ('shining', 1)]
